# Notebook for regenerating tf-idf vectors from wiki articles
Unfortunatly, authors didn't provide tfidf vectorizer with vocab, so it's impossible to do the same transformation to input text in AttnGAN code. The best way to fix it that I can see is to recalculate tf-idf vectors. After that I'll be able to train a new ZSL_GAN encoder and use it with the same transformations in AttGan.

In [1]:
import os
import nltk
import string
import numpy as np

from text_processor import TextProcessor

In [2]:
# !pip install --user nltk

# nltk.download('punkt')
# nltk.download('stopwords')

In [3]:
final_descriptions = ['' for i in range(200)]
text_processor = TextProcessor()

wiki_articles_folder_path = 'data/Raw_Wiki_Articles/CUBird_Sentences'
for class_file in os.listdir(wiki_articles_folder_path):
    if class_file[0] not in string.digits:
        continue

    class_id = int(class_file.split('.')[0])  # 1-indexed!
    
    text = ''
    for line in open(os.path.join(wiki_articles_folder_path, class_file), 'r').readlines():
        line = line.strip()
        start_id = 0
        while start_id < len(line) and line[start_id] in string.digits:
            start_id += 1
            
        if start_id + 1 < len(line):
            text += line[start_id + 1:]
    
    final_descriptions[class_id - 1] = text_processor.preprocess_text(text)

In [4]:
tf_idf = text_processor.get_tfidf(final_descriptions, 'out/tf_idf_vectorizer.pkl', False)

In [5]:
# original article mentioned 7551 for CUB, so it's pretty close
print(tf_idf.shape)

(200, 7549)


In [6]:
np.save('data/CUB2011/new_tfidf.npy', tf_idf)